# Q2

En esta parte, la tarea consiste en llamar a MongoDB para realizar geoqueries y representar los datos geoespaciales con folium, mostrando en un mapa la geolocalizacion de la nueva oficina.

In [32]:
# Tu codigo aqui
import pymongo
from pymongo import MongoClient
import folium
from folium import plugins
import pandas as pd
from bson.json_util import loads

In [33]:
client = MongoClient('mongodb://localhost:27017/')
db = client.db_geoloc
colec = db.geop

In [34]:
colec.insert_many([loads(one) for one in open('geop.json','r')][0]);

In [35]:
df = pd.read_json('geop.json')
df

,Geolocation,address1,address2,city,country_code,description,latitude,longitude,name,number_of_employees,state_code,zip_code
0,"{'type': 'Point', 'coordinates': [-73.723999, ...",New Orchard Road,,Armonk,USA,Corporate Headquarters,41.109534,-73.723999,IBM,388000,NY,10504
1,"{'type': 'Point', 'coordinates': [24.827224, 6...",Keilalahdentie 2-4,"P.O. Box 226, FIN-00045 Nokia Group",,FIN,headquarter,60.170514,24.827224,Nokia,125000,None,FIN-00045
2,"{'type': 'Point', 'coordinates': [-75.1652518,...",1701 JFK Blvd,,Philadelphia,USA,One Comcast Center,39.952576,-75.165252,Comcast,100000,PA,19103
3,"{'type': 'Point', 'coordinates': [-71.099692, ...",350 Massachusetts Ave,,Cambridge,USA,World Headquarters,42.362360,-71.099692,MetaCarta,99999,MA,02139
4,"{'type': 'Point', 'coordinates': [-122.128585,...",1 Microsoft Way,,Redmond,USA,World Headquarters,47.643583,-122.128585,Microsoft,90000,WA,
5,"{'type': 'Point', 'coordinates': [-116.209356,...",500 S 10th St.,,Boise,USA,Boise Development Center,43.613184,-116.209356,Cisco,63000,ID,83702
6,"{'type': 'Point', 'coordinates': [-121.95377, ...",170 West Tasman Dr.,,San Jose,USA,Headquarters,37.408802,-121.953770,Nortel Networks,30307,CA,95134
7,"{'type': 'Point', 'coordinates': [-79.5589112,...",195 The West Mall,,"Toronto, ON",CAN,,43.626967,-79.558911,The Walt Disney Company,25000,None,M9C 5K1
8,"{'type': 'Point', 'coordinates': [-118.324987,...",500 S. Buena Vista St.,,Burbank,USA,None,34.156802,-118.324987,Avaya,18000,CA,91521
9,"{'type': 'Point', 'coordinates': [-121.977333,...",4655 Great America Parkway,,Santa Clara,USA,California,37.398076,-121.977333,Experian,15500,CA,95054


In [36]:
db.geop.create_index([('Geolocation', pymongo.GEOSPHERE)])

'Geolocation_2dsphere'

In [37]:
def findNear(location, ratio=2000):
    return colec.find(
                        {'Geolocation':
                             {'$near':{'$geometry':location, '$maxDistance':ratio}}
                        }
                        )
df['nearest'] = df['Geolocation'].apply(lambda x: list(findNear(x)))

In [70]:
points = folium.Map([41.109534,-73.72399],tiles='Stamen Toner', zoom_start=10) 
points

In [71]:
for p in df.index:
    folium.Marker([df.loc[p,'latitude'], df.loc[p,'longitude']],icon=folium.Icon(icon='cloud'),
                popup=df.loc[p,'name']).add_to(points)

In [72]:
points